In [22]:
import numpy as np
import pandas as pd
from tqdm import tqdm

pd.set_option('display.max_columns', 40)
match_id = 2499719
match_events = pd.read_pickle(f'data/refined_events/England/{match_id}.pkl')
#슈팅 횟수
shot_records = match_events[
    (match_events['event_type'] == 'Shot') |
    (match_events['sub_event_type'].isin(['Free kick shot', 'Penalty']))
]
shots = shot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
shots.name = 'total_shots'
#패스 횟수
pass_records = match_events[
    (match_events['event_type'] == 'Pass') |
    (match_events['sub_event_type'].isin(['Free kick', 'Free kick cross', 'corner']))
]
passes = pass_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
passes.name = 'total_passes'
#파울 횟수
foul_records = match_events[match_events['event_type'] == 'Foul']
fouls = foul_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
fouls.name = 'fouls'
#오프사이드 횟수
offside_records = match_events[match_events['event_type'] == 'Offside']
offsides = offside_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
offsides.name = 'offsides'
#유효 슈팅 횟수 + 신체 부위별 슈팅 횟수
acc_shot_records = shot_records[shot_records['tags'].apply(lambda x: 'Accurate' in x)]
acc_shots = acc_shot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
acc_shots.name = 'shots_on_target'
rshot_records = shot_records[shot_records['tags'].apply(lambda x: 'Right foot' in x)]
rshots = rshot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
rshots.name = 'rfoot_shots'
lshot_records = shot_records[shot_records['tags'].apply(lambda x: 'Left foot' in x)]
lshots = lshot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
lshots.name = 'lfoot_shots'
hshot_records = shot_records[shot_records['tags'].apply(lambda x: 'Head/body' in x)]
hshots = hshot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
hshots.name = 'header_shots'
shot_stats_list = [shots, acc_shots, rshots, lshots, hshots]
shot_stats = pd.concat(shot_stats_list, axis=1).fillna(0).astype(int)
#득점/도움/자책골 횟수
goal_records = match_events[match_events['tags'].apply(lambda x: 'Goal' in x)]
goals = goal_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
goals.name = 'goals' #골
assist_records = match_events[match_events['tags'].apply(lambda x: 'Assist' in x)]
assists = assist_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
assists.name = 'assists' #도움
own_goal_records = match_events[match_events['tags'].apply(lambda x: 'Own goal' in x)]
own_goals = own_goal_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
own_goals.name = 'own_goals' #자책골
goal_stats_list = [goals, assists, own_goals]
goal_stats = pd.concat(goal_stats_list, axis=1).fillna(0).astype(int)
#패스 성공률
acc_pass_records = pass_records[pass_records['tags'].apply(lambda x: 'Accurate' in x)]
acc_passes = acc_pass_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
acc_passes.name = 'acc_passes'
pass_stats = pd.concat([passes, acc_passes], axis=1).fillna(0).astype(int)
pass_stats['pass_accuracy'] = (pass_stats['acc_passes'] / pass_stats['total_passes']).round(2)
#경고/퇴장 횟수
yellow_records = foul_records[foul_records['tags'].apply(lambda x: 'Yellow card' in x)]
yellows = yellow_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
yellows.name = 'yellow_cards'
red_records = foul_records[foul_records['tags'].apply(lambda x: 'Red card' in x)]
reds = red_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
reds.name = 'red_cards'
foul_stats = pd.concat([fouls, offsides, yellows, reds], axis=1).fillna(0).astype(int)
#경기 내 선수별 기록 정리표
player_stats = pd.concat([goal_stats, shot_stats, foul_stats, pass_stats], axis=1, sort=True)
player_stats = player_stats.fillna(0).reset_index()
for col in player_stats.columns[4:]:
    if col != 'pass_accuracy':
        player_stats[col] = player_stats[col].astype(int)
#선수교체 및 퇴장 기록 필터링
player_change_records = match_events[
    (match_events['event_type'] == 'Substitution') |
    (match_events['tags'].apply(lambda x: 'Red card' in x))
]
#선발 선수 추출
in_players = player_change_records[player_change_records['sub_event_type'] == 'Player in']['player_id'].tolist()
player_ids = [p for p in match_events['player_id'].unique() if not p in in_players]
#선수 교체 및 퇴장 기록을 활용한 페이즈 구분 
period_durations = match_events.groupby('period')['time'].max()
phase_record_list = []
phase = 1

for period in period_durations.index:
    change_times = player_change_records[player_change_records['period'] == period]['time'].unique().tolist()
    change_times.append(period_durations[period])
    if 0 not in change_times:
        change_times = [0] + change_times

    for i in range(len(change_times[:-1])):
        moment_records = player_change_records[
            (player_change_records['period'] == period) &
            (player_change_records['time'] == change_times[i])
        ]

        for _, record in moment_records.iterrows():
            if record['sub_event_type'] == 'Player out' or record['event_type'] == 'Foul':
                player_ids.remove(record['player_id'])
            else:
                player_ids.append(record['player_id'])

        phase_record = {
            'phase': phase,
            'period': period,
            'start_time': change_times[i],
            'end_time': change_times[i+1],
            'duration': change_times[i+1] - change_times[i],
            'player_ids': player_ids.copy()
        }
        phase += 1

        phase_record_list.append(phase_record)

phase_records = pd.DataFrame(phase_record_list).set_index('phase')
#선수별 각 페이즈 출전 여부 판단
player_ids = np.sort(match_events['player_id'].unique())
for player_id in player_ids:
    phase_records[player_id] = 0

for phase in phase_records.index:
    for player_id in phase_records.at[phase, 'player_ids']:
        phase_records.at[phase, player_id] = 1

phase_records = phase_records[np.concatenate([phase_records.columns[:4], player_ids])]
#선수별 출전 시간 산출
playing_times = pd.Series(index=player_ids, dtype='float')
for player_id in player_ids:
    playing_times[player_id] = phase_records[phase_records[player_id] == 1]['duration'].sum().round(1)
playing_times = playing_times.reset_index()
playing_times.columns = ['player_id', 'playing_time']
#출전 시간 정보 추가
player_stats = pd.merge(player_stats, playing_times)
player_stats

,team_id,team_name,player_id,player_name,goals,assists,own_goals,total_shots,shots_on_target,rfoot_shots,lfoot_shots,header_shots,fouls,offsides,yellow_cards,red_cards,total_passes,acc_passes,pass_accuracy,playing_time
0,1609,Arsenal,3319,M. Özil,0,0,0,3,0,0,3,0,0,2,0,0,82,69,0.84,5859.2
1,1609,Arsenal,3560,Nacho Monreal,0,0,0,0,0,0,0,0,0,0,0,0,58,56,0.97,5859.2
2,1609,Arsenal,7868,A. Oxlade-Chamberlain,0,0,0,6,2,5,1,0,1,0,0,0,55,40,0.73,5859.2
3,1609,Arsenal,7870,A. Ramsey,1,0,0,4,1,2,0,2,0,0,0,0,9,7,0.78,1746.6
4,1609,Arsenal,7879,T. Walcott,0,0,0,0,0,0,0,0,0,1,0,0,2,1,0.50,1266.6
5,1609,Arsenal,7882,P. Čech,0,0,0,0,0,0,0,0,0,0,0,0,14,13,0.93,5859.2
6,1609,Arsenal,7945,D. Welbeck,1,0,0,3,1,2,1,0,1,1,0,0,20,16,0.80,4592.6
7,1609,Arsenal,14869,S. Kolašinac,0,1,0,1,1,0,1,0,1,0,0,0,71,58,0.82,5859.2
8,1609,Arsenal,25413,A. Lacazette,1,0,0,3,2,2,1,0,2,2,0,0,20,15,0.75,5859.2
9,1609,Arsenal,26010,O. Giroud,1,0,0,1,1,0,0,1,0,0,0,0,6,6,1.00,1746.6


In [6]:
import numpy as np
import pandas as pd
from tqdm import tqdm

pd.set_option('display.max_columns', 40)

def generate_player_stats(match_id):
    # Data loading
    match_events = pd.read_pickle(f'Analysis/data/refined_events/England/{match_id}.pkl')
    match_events = match_events[match_events['period'] != 'P']

    # Goal stats
    goal_records = match_events[match_events['tags'].apply(lambda x: 'Goal' in x)]
    goals = goal_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    goals.name = 'goals'

    own_goal_records = match_events[match_events['tags'].apply(lambda x: 'Own goal' in x)]
    own_goals = own_goal_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    own_goals.name = 'own_goals'

    assist_records = match_events[match_events['tags'].apply(lambda x: 'Assist' in x)]
    assists = assist_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    assists.name = 'assists'

    goal_stats_list = [goals, assists, own_goals]
    goal_stats = pd.concat(goal_stats_list, axis=1).fillna(0).astype(int)

    # Shot stats
    shot_records = match_events[
        (match_events['event_type'] == 'Shot') |
        (match_events['sub_event_type'].isin(['Free kick shot', 'Penalty']))
    ]
    shots = shot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    shots.name = 'total_shots'

    acc_shot_records = shot_records[shot_records['tags'].apply(lambda x: 'Accurate' in x)]
    acc_shots = acc_shot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    acc_shots.name = 'shots_on_target'

    rshot_records = shot_records[shot_records['tags'].apply(lambda x: 'Right foot' in x)]
    rshots = rshot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    rshots.name = 'rfoot_shots'

    lshot_records = shot_records[shot_records['tags'].apply(lambda x: 'Left foot' in x)]
    lshots = lshot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    lshots.name = 'lfoot_shots'

    hshot_records = shot_records[shot_records['tags'].apply(lambda x: 'Head/body' in x)]
    hshots = hshot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    hshots.name = 'header_shots'

    shot_stats_list = [shots, acc_shots, rshots, lshots, hshots]
    shot_stats = pd.concat(shot_stats_list, axis=1).fillna(0).astype(int)

    # Foul stats
    foul_records = match_events[match_events['event_type'] == 'Foul']
    fouls = foul_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    fouls.name = 'fouls'

    offside_records = match_events[match_events['event_type'] == 'Offside']
    offsides = offside_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    offsides.name = 'offsides'

    yellow_records = foul_records[foul_records['tags'].apply(lambda x: 'Yellow card' in x)]
    yellows = yellow_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    yellows.name = 'yellow_cards'

    red_records = foul_records[foul_records['tags'].apply(lambda x: 'Red card' in x)]
    reds = red_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    reds.name = 'red_cards'

    foul_stats = pd.concat([fouls, offsides, yellows, reds], axis=1).fillna(0).astype(int)

    # Pass stats
    pass_records = match_events[
        (match_events['event_type'] == 'Pass') |
        (match_events['sub_event_type'].isin(['Free kick', 'Free kick cross', 'corner']))
    ]
    passes = pass_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    passes.name = 'total_passes'

    acc_pass_records = pass_records[pass_records['tags'].apply(lambda x: 'Accurate' in x)]
    acc_passes = acc_pass_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    acc_passes.name = 'acc_passes'

    pass_stats = pd.concat([passes, acc_passes], axis=1).fillna(0).astype(int)
    pass_stats['pass_accuracy'] = (pass_stats['acc_passes'] / pass_stats['total_passes']).round(2)

    # Playing time
    player_change_records = match_events[
        (match_events['event_type'] == 'Substitution') |
        (match_events['tags'].apply(lambda x: 'Red card' in x))
    ]
    in_players = player_change_records[player_change_records['sub_event_type'] == 'Player in']['player_id'].tolist()
    player_ids = [p for p in match_events['player_id'].unique() if not p in in_players]

    period_durations = match_events.groupby('period')['time'].max()
    phase_record_list = []
    phase = 1

    for period in period_durations.index:
        change_times = player_change_records[player_change_records['period'] == period]['time'].unique().tolist()
        change_times.append(period_durations[period])
        if 0 not in change_times:
            change_times = [0] + change_times

        for i in range(len(change_times[:-1])):
            moment_records = player_change_records[
                (player_change_records['period'] == period) &
                (player_change_records['time'] == change_times[i])
            ]

            for _, record in moment_records.iterrows():
                if record['sub_event_type'] == 'Player out' or record['event_type'] == 'Foul':
                    player_ids.remove(record['player_id'])
                else:
                    player_ids.append(record['player_id'])

            phase_record = {
                'phase': phase,
                'period': period,
                'start_time': change_times[i],
                'end_time': change_times[i+1],
                'duration': change_times[i+1] - change_times[i],
                'player_ids': player_ids.copy()
            }
            phase += 1

            phase_record_list.append(phase_record)

    phase_records = pd.DataFrame(phase_record_list).set_index('phase')

    player_ids = np.sort(match_events['player_id'].unique())
    for player_id in player_ids:
        phase_records[player_id] = 0

    for phase in phase_records.index:
        for player_id in phase_records.at[phase, 'player_ids']:
            phase_records.at[phase, player_id] = 1

    phase_records = phase_records[np.concatenate([phase_records.columns[:4], player_ids])]

    playing_times = pd.Series(index=player_ids, dtype='float')
    for player_id in player_ids:
        playing_times[player_id] = phase_records[phase_records[player_id] == 1]['duration'].sum().round(1)
    playing_times = playing_times.reset_index()
    playing_times.columns = ['player_id', 'playing_time']

    # Concatenation
    player_stats = pd.concat([goal_stats, shot_stats, foul_stats, pass_stats], axis=1, sort=True).fillna(0)
    for col in player_stats.columns:
        if col != 'pass_accuracy':
            player_stats[col] = player_stats[col].astype(int)

    player_stats = pd.merge(player_stats.reset_index(), playing_times)
    player_stats['match_id'] = match_id

    cols = player_stats.columns.tolist()
    cols = ['match_id'] + cols[:4] + ['playing_time'] + cols[4:-2]
    return player_stats[cols]


dataset_name = 'England'
match_df = pd.read_csv(f'Analysis/data/refined_events/{dataset_name}/matches.csv', index_col=0, encoding='utf-8-sig')
match_df


stats_list = []

for match_id in tqdm(match_df.index):
    match_player_stats = generate_player_stats(match_id)
    stats_list.append(match_player_stats)

player_stats = pd.concat(stats_list, ignore_index=True)
player_stats

grouped = player_stats.groupby(['team_id', 'team_name', 'player_id', 'player_name'])

player_stats_accum = grouped[player_stats.columns[5:-1]].sum()
player_stats_accum['pass_accuracy'] = (player_stats_accum['acc_passes'] / player_stats_accum['total_passes']).round(2)
player_stats_accum['matches'] = grouped['match_id'].count()

player_stats_accum = player_stats_accum[['matches'] + player_stats.columns[5:-1].tolist()].reset_index()
player_stats_accum

player_stats_accum.sort_values('goals', ascending=False, ignore_index=True)[:10]

# 결과를 JSON으로 출력
player_stats_accum.sort_values('goals', ascending=False, ignore_index=True)[:10].to_json(orient='records')


FileNotFoundError: [Errno 2] No such file or directory: 'Analysis/data/refined_events/England/matches.csv'

In [7]:
import numpy as np
import pandas as pd
from tqdm import tqdm

pd.set_option('display.max_columns', 40)

def generate_player_stats(match_id):
    # Data loading
    match_events = pd.read_pickle(f'data/refined_events/England/{match_id}.pkl')
    match_events = match_events[match_events['period'] != 'P']

    # Goal stats
    goal_records = match_events[match_events['tags'].apply(lambda x: 'Goal' in x)]
    goals = goal_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    goals.name = 'goals'

    own_goal_records = match_events[match_events['tags'].apply(lambda x: 'Own goal' in x)]
    own_goals = own_goal_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    own_goals.name = 'own_goals'

    assist_records = match_events[match_events['tags'].apply(lambda x: 'Assist' in x)]
    assists = assist_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    assists.name = 'assists'

    goal_stats_list = [goals, assists, own_goals]
    goal_stats = pd.concat(goal_stats_list, axis=1).fillna(0).astype(int)
    goal_stats

In [10]:
import numpy as np
import pandas as pd
from tqdm import tqdm
match_id = 2499719
match_events = pd.read_pickle(f'data/refined_events/England/{match_id}.pkl')
dataset_name = 'England'
match_df = pd.read_csv(f'data/refined_events/{dataset_name}/matches.csv', index_col=0, encoding='utf-8-sig')
match_df
def generate_player_stats(match_id):
    # Data loading
    match_events = pd.read_pickle(f'data/refined_events/England/{match_id}.pkl')
    match_events = match_events[match_events['period'] != 'P']

    # Goal stats
    goal_records = match_events[match_events['tags'].apply(lambda x: 'Goal' in x)]
    goals = goal_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    goals.name = 'goals'

    own_goal_records = match_events[match_events['tags'].apply(lambda x: 'Own goal' in x)]
    own_goals = own_goal_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    own_goals.name = 'own_goals'

    assist_records = match_events[match_events['tags'].apply(lambda x: 'Assist' in x)]
    assists = assist_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    assists.name = 'assists'

    goal_stats_list = [goals, assists, own_goals]
    goal_stats = pd.concat(goal_stats_list, axis=1).fillna(0).astype(int)

    # Shot stats
    shot_records = match_events[
        (match_events['event_type'] == 'Shot') |
        (match_events['sub_event_type'].isin(['Free kick shot', 'Penalty']))
    ]
    shots = shot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    shots.name = 'total_shots'

    acc_shot_records = shot_records[shot_records['tags'].apply(lambda x: 'Accurate' in x)]
    acc_shots = acc_shot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    acc_shots.name = 'shots_on_target'

    rshot_records = shot_records[shot_records['tags'].apply(lambda x: 'Right foot' in x)]
    rshots = rshot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    rshots.name = 'rfoot_shots'

    lshot_records = shot_records[shot_records['tags'].apply(lambda x: 'Left foot' in x)]
    lshots = lshot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    lshots.name = 'lfoot_shots'

    hshot_records = shot_records[shot_records['tags'].apply(lambda x: 'Head/body' in x)]
    hshots = hshot_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    hshots.name = 'header_shots'

    shot_stats_list = [shots, acc_shots, rshots, lshots, hshots]
    shot_stats = pd.concat(shot_stats_list, axis=1).fillna(0).astype(int)

    # Foul stats
    foul_records = match_events[match_events['event_type'] == 'Foul']
    fouls = foul_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    fouls.name = 'fouls'

    offside_records = match_events[match_events['event_type'] == 'Offside']
    offsides = offside_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    offsides.name = 'offsides'

    yellow_records = foul_records[foul_records['tags'].apply(lambda x: 'Yellow card' in x)]
    yellows = yellow_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    yellows.name = 'yellow_cards'

    red_records = foul_records[foul_records['tags'].apply(lambda x: 'Red card' in x)]
    reds = red_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    reds.name = 'red_cards'

    foul_stats = pd.concat([fouls, offsides, yellows, reds], axis=1).fillna(0).astype(int)

    # Pass stats
    pass_records = match_events[
        (match_events['event_type'] == 'Pass') |
        (match_events['sub_event_type'].isin(['Free kick', 'Free kick cross', 'corner']))
    ]
    passes = pass_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    passes.name = 'total_passes'

    acc_pass_records = pass_records[pass_records['tags'].apply(lambda x: 'Accurate' in x)]
    acc_passes = acc_pass_records.groupby(['team_id', 'team_name', 'player_id', 'player_name'])['event_id'].count()
    acc_passes.name = 'acc_passes'

    pass_stats = pd.concat([passes, acc_passes], axis=1).fillna(0).astype(int)
    pass_stats['pass_accuracy'] = (pass_stats['acc_passes'] / pass_stats['total_passes']).round(2)

    # Playing time
    player_change_records = match_events[
        (match_events['event_type'] == 'Substitution') |
        (match_events['tags'].apply(lambda x: 'Red card' in x))
    ]
    in_players = player_change_records[player_change_records['sub_event_type'] == 'Player in']['player_id'].tolist()
    player_ids = [p for p in match_events['player_id'].unique() if not p in in_players]

    period_durations = match_events.groupby('period')['time'].max()
    phase_record_list = []
    phase = 1

    for period in period_durations.index:
        change_times = player_change_records[player_change_records['period'] == period]['time'].unique().tolist()
        change_times.append(period_durations[period])
        if 0 not in change_times:
            change_times = [0] + change_times

        for i in range(len(change_times[:-1])):
            moment_records = player_change_records[
                (player_change_records['period'] == period) &
                (player_change_records['time'] == change_times[i])
            ]

            for _, record in moment_records.iterrows():
                if record['sub_event_type'] == 'Player out' or record['event_type'] == 'Foul':
                    player_ids.remove(record['player_id'])
                else:
                    player_ids.append(record['player_id'])

            phase_record = {
                'phase': phase,
                'period': period,
                'start_time': change_times[i],
                'end_time': change_times[i+1],
                'duration': change_times[i+1] - change_times[i],
                'player_ids': player_ids.copy()
            }
            phase += 1

            phase_record_list.append(phase_record)

    phase_records = pd.DataFrame(phase_record_list).set_index('phase')

    player_ids = np.sort(match_events['player_id'].unique())
    for player_id in player_ids:
        phase_records[player_id] = 0

    for phase in phase_records.index:
        for player_id in phase_records.at[phase, 'player_ids']:
            phase_records.at[phase, player_id] = 1

    phase_records = phase_records[np.concatenate([phase_records.columns[:4], player_ids])]

    playing_times = pd.Series(index=player_ids, dtype='float')
    for player_id in player_ids:
        playing_times[player_id] = phase_records[phase_records[player_id] == 1]['duration'].sum().round(1)
    playing_times = playing_times.reset_index()
    playing_times.columns = ['player_id', 'playing_time']

    # Concatenation
    player_stats = pd.concat([goal_stats, shot_stats, foul_stats, pass_stats], axis=1, sort=True).fillna(0)
    for col in player_stats.columns:
        if col != 'pass_accuracy':
            player_stats[col] = player_stats[col].astype(int)

    player_stats = pd.merge(player_stats.reset_index(), playing_times)
    player_stats['match_id'] = match_id

    cols = player_stats.columns.tolist()
    cols = ['match_id'] + cols[:4] + ['playing_time'] + cols[4:-2]
    return player_stats[cols]
stats_list = []

for match_id in tqdm(match_df.index):
    match_player_stats = generate_player_stats(match_id)
    stats_list.append(match_player_stats)

player_stats = pd.concat(stats_list, ignore_index=True)
player_stats


100%|██████████| 380/380 [00:21<00:00, 17.92it/s]


,match_id,team_id,team_name,player_id,player_name,playing_time,goals,assists,own_goals,total_shots,shots_on_target,rfoot_shots,lfoot_shots,header_shots,fouls,offsides,yellow_cards,red_cards,total_passes,acc_passes,pass_accuracy
0,2499719,1609,Arsenal,3319,M. Özil,5859.2,0,0,0,3,0,0,3,0,0,2,0,0,82,69,0.84
1,2499719,1609,Arsenal,3560,Nacho Monreal,5859.2,0,0,0,0,0,0,0,0,0,0,0,0,58,56,0.97
2,2499719,1609,Arsenal,7868,A. Oxlade-Chamberlain,5859.2,0,0,0,6,2,5,1,0,1,0,0,0,55,40,0.73
3,2499719,1609,Arsenal,7870,A. Ramsey,1746.6,1,0,0,4,1,2,0,2,0,0,0,0,9,7,0.78
4,2499719,1609,Arsenal,7879,T. Walcott,1266.6,0,0,0,0,0,0,0,0,0,1,0,0,2,1,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10347,2500089,1659,AFC Bournemouth,62224,R. Fraser,5787.4,0,0,0,0,0,0,0,0,1,0,0,0,31,23,0.74
10348,2500089,1659,AFC Bournemouth,134102,N. Aké,5787.4,0,0,0,0,0,0,0,0,1,0,0,0,31,25,0.81
10349,2500089,1659,AFC Bournemouth,239411,T. Mings,3906.2,0,0,0,1,0,0,1,0,0,0,0,0,45,38,0.84
10350,2500089,1659,AFC Bournemouth,245813,L. Mousset,3546.2,0,0,0,2,1,2,0,0,1,0,0,0,23,15,0.65
